## DeviceArray properties

In [ ]:
import numpy as np
import jax.numpy as jnp

In [ ]:
np.array([1, 42, 31337])

array([    1,    42, 31337])

In [ ]:
jnp.array([1, 42, 31337])

DeviceArray([    1,    42, 31337], dtype=int32)

In [ ]:
np.sum([1, 42, 31337])

31380

In [ ]:
try:
  jnp.sum([1, 42, 31337])
except TypeError as e:
  print(e)

sum requires ndarray or scalar arguments, got <class 'list'> at position 0.


In [ ]:
jnp.sum(jnp.array([1, 42, 31337]))

DeviceArray(31380, dtype=int32)

In [ ]:
arr = jnp.array([1, 42, 31337])

In [ ]:
arr.ndim

1

In [ ]:
arr.shape

(3,)

In [ ]:
arr.dtype

dtype('int32')

In [ ]:
arr.size

3

In [ ]:
arr.nbytes

12

## Devices

In [ ]:
import jax

In [ ]:
jax.devices()

[GpuDevice(id=0, process_index=0)]

In [ ]:
jax.devices('cpu')

[CpuDevice(id=0)]

In [ ]:
jax.device_count('gpu')

1

In [ ]:
jax.local_devices()

[GpuDevice(id=0, process_index=0)]

In [ ]:
arr = jnp.array([1, 42, 31337])

In [ ]:
arr.device()

GpuDevice(id=0, process_index=0)

In [ ]:
arr_cpu = jax.device_put(arr, jax.devices('cpu')[0])

In [ ]:
arr_cpu.device()

CpuDevice(id=0)

In [ ]:
arr.device()

GpuDevice(id=0, process_index=0)

In [ ]:
arr_host = jax.device_get(arr)

In [ ]:
type(arr_host)

numpy.ndarray

In [ ]:
arr_host

array([    1,    42, 31337], dtype=int32)

In [ ]:
arr + arr_cpu

DeviceArray([    2,    84, 62674], dtype=int32)

In [ ]:
arr_gpu = jax.device_put(arr, jax.devices('gpu')[0])

In [ ]:
arr_gpu.device()

GpuDevice(id=0, process_index=0)

In [ ]:
try:
  arr_gpu + arr_cpu
except ValueError as e:
  print(e)

primitive arguments must be colocated on the same device (C++ jax.jit). Arguments are on devices: TFRT_CPU_0 and gpu:0


## Asyncronous dispatch

In [ ]:
import jax

In [ ]:
a = jnp.array(range(1000000)).reshape((1000,1000))

In [ ]:
a.shape

(1000, 1000)

In [ ]:
a.device()

GpuDevice(id=0, process_index=0)

In [ ]:
%time x = jnp.dot(a,a)

CPU times: user 757 µs, sys: 0 ns, total: 757 µs
Wall time: 770 µs


In [ ]:
%time x = jnp.dot(a,a).block_until_ready()

CPU times: user 1.34 ms, sys: 65 µs, total: 1.41 ms
Wall time: 4.33 ms


In [ ]:
%time x = np.asarray(jnp.dot(a,a))

In [ ]:
a_cpu = jax.device_put(a, jax.devices('cpu')[0])

In [ ]:
a_cpu.device()

CpuDevice(id=0)

In [ ]:
%time x = jnp.dot(a_cpu,a_cpu).block_until_ready()

CPU times: user 272 ms, sys: 0 ns, total: 272 ms
Wall time: 150 ms


## Using TPU

In [ ]:
# in order to use TPU you have to run this cell before importing JAX
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [ ]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

tpu


In [ ]:
import jax

In [ ]:
jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [ ]:
import jax.numpy as jnp

In [ ]:
a = jnp.array(range(1000000)).reshape((1000,1000))

In [ ]:
a.device()

TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)

In [ ]:
%time x = jnp.dot(a,a)

CPU times: user 107 ms, sys: 256 ms, total: 363 ms
Wall time: 726 ms


In [ ]:
%time x = jnp.dot(a,a).block_until_ready()

CPU times: user 1.32 ms, sys: 2.63 ms, total: 3.94 ms
Wall time: 4.19 ms


In [ ]:
import os
from tensorflow.python.profiler import profiler_client

tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
print(profiler_client.monitor(tpu_profile_service_address, 100, 2))

  Timestamp: 08:53:30
  TPU type: TPU v2
  Utilization of TPU Matrix Units (higher is better): 0.000%




## GPU diagnostics

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
#!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


## Immutability

In [ ]:
import numpy as np
import jax.numpy as jnp

In [ ]:
a_jnp = jnp.array(range(10))
a_np  = np.array(range(10))

In [ ]:
a_jnp

DeviceArray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [ ]:
a_np[5], a_jnp[5]

(5, DeviceArray(5, dtype=int32))

In [ ]:
a_np[5] = 100

In [ ]:
a_np[5]

100

In [ ]:
try:
  a_jnp[5] = 100
except TypeError as e:
  print(e)

'<class 'jax._src.device_array._DeviceArray'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html


In [ ]:
a_jnp = a_jnp.at[5].set(100)

In [ ]:
a_jnp[5]

DeviceArray(100, dtype=int32)

In [ ]:
a_jnp[42]

DeviceArray(9, dtype=int32)

In [ ]:
a_jnp.at[42].get()

DeviceArray(9, dtype=int32)

In [ ]:
a_jnp.at[42].get(mode='clip')

DeviceArray(9, dtype=int32)

In [ ]:
a_jnp.at[42].get(mode='drop')

DeviceArray(-2147483648, dtype=int32)

In [ ]:
a_jnp.at[42].get(mode='fill', fill_value=-1)

DeviceArray(-1, dtype=int32)

In [ ]:
a_jnp = a_jnp.at[42].set(100)
a_jnp

DeviceArray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [ ]:
a_jnp = a_jnp.at[42].set(100, mode='clip')
a_jnp

DeviceArray([  0,   1,   2,   3,   4,   5,   6,   7,   8, 100], dtype=int32)

## Working with float64

In [ ]:
# this only works on startup!
from jax.config import config
config.update("jax_enable_x64", True)

In [ ]:
import jax
import jax.numpy as jnp

In [ ]:
# this will not work on TPU backend. Try using CPU or GPU.
x = jnp.array(range(10), dtype=jnp.float64)
x.dtype

dtype('float64')

In [ ]:
x.device()

GpuDevice(id=0, process_index=0)

In [ ]:
xc = jax.device_put(x, jax.devices('cpu')[0])

In [ ]:
xc.device()

CpuDevice(id=0)

In [ ]:
xc.dtype

dtype('float64')

In [ ]:
xb16 = jnp.array(range(10), dtype=jnp.bfloat16)
xb16.dtype

dtype(bfloat16)

In [ ]:
xb16.nbytes

20

In [ ]:
x16 = jnp.array(range(10), dtype=jnp.float16)
x16.dtype

dtype('float16')

In [ ]:
x16.nbytes

20

In [ ]:
xb16+x16

DeviceArray([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.], dtype=float32)

In [ ]:
xb16+xb16

DeviceArray([0, 2, 4, 6, 8, 10, 12, 14, 16, 18], dtype=bfloat16)

## jax.numpy & jax.lax

In [ ]:
import jax.numpy as jnp
from jax import lax
from jax import random

In [ ]:
jnp.add(42, 42.0)

DeviceArray(84., dtype=float32, weak_type=True)

In [ ]:
jnp.add(42.0, 42.0)

DeviceArray(84., dtype=float32, weak_type=True)

In [ ]:
try:
   lax.add(42, 42.0)
except TypeError as e:
  print(e)

lax.add requires arguments to have the same dtypes, got int32, float32. (Tip: jnp.add is a similar function that does automatic type promotion on inputs).


In [ ]:
lax.add(jnp.float32(42), 42.0)

DeviceArray(84., dtype=float32)

In [ ]:
def random_augmentation(image, augmentations, rng_key):
   '''A function that applies a random transformation to an image'''
   augmentation_index = random.randint(key=rng_key, minval=0, maxval=len(augmentations), shape=())
   augmented_image = lax.switch(augmentation_index, augmentations, image)
   return augmented_image

In [ ]:
add_noise_func = lambda x: x+10
horizontal_flip_func = lambda x: x+1
rotate_func = lambda x: x+2
adjust_colors_func = lambda x: x+3

augmentations = [
   add_noise_func,
   horizontal_flip_func,
   rotate_func,
   adjust_colors_func
]


In [ ]:
image = jnp.array(range(100))

In [ ]:
random_augmentation(image, augmentations, random.PRNGKey(211))

DeviceArray([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
              13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
              25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,
              37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
              49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,
              61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
              73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
              85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
              97,  98,  99, 100], dtype=int32)